In [5]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import ta  # Technical Analysis library for additional indicators


In [6]:
# Download the stock data (Apple Inc. as an example)
stock_data = yf.download('AAPL', start='2010-01-01', end='2021-12-31')

# Display the first few rows of the stock data
stock_data.head()


C:\Users\Muhammad Bin Mehmood\AppData\Local\Temp\ipykernel_9028\3269103827.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download('AAPL', start='2010-01-01', end='2021-12-31')
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2010-01-04,6.431896,6.446623,6.382908,6.414465,493729600
2010-01-05,6.443016,6.479382,6.409054,6.449628,601904800
2010-01-06,6.340531,6.468562,6.333919,6.443016,552160000
2010-01-07,6.328811,6.371488,6.282828,6.363974,477131200
2010-01-08,6.370886,6.371487,6.283128,6.320395,447610800


In [10]:
# Add Technical Indicators (e.g., Moving Averages, RSI, Bollinger Bands)
stock_data['SMA_50'] = stock_data['Close'].rolling(window=50).mean()
stock_data['SMA_200'] = stock_data['Close'].rolling(window=200).mean()

# RSI: Extract Close column as a 1-dimensional series
stock_data['RSI'] = ta.momentum.RSIIndicator(stock_data['Close'].squeeze()).rsi()

# Bollinger Bands: Compute Upper and Lower Bands
stock_data['Upper Band'], stock_data['Lower Band'] = ta.volatility.BollingerBands(stock_data['Close']).bollinger_hband(), ta.volatility.BollingerBands(stock_data['Close']).bollinger_lband()

# Volume Moving Average: 50-day Moving Average of Volume
stock_data['Volume_MA'] = stock_data['Volume'].rolling(window=50).mean()

# Fill any missing values created by the indicators
stock_data.fillna(method='bfill', inplace=True)

# Display the first few rows with technical indicators
stock_data.head()


ValueError: Data must be 1-dimensional, got ndarray of shape (3020, 1) instead